In [17]:
import ast

import numpy as np
import pandas as pd

from config import ROOT_DIR

In [18]:
df = pd.read_csv(ROOT_DIR + "/data/raw/lichess_db_standard_rated_2018-02/data_0.csv")

In [97]:
df[["BaseTime", "IncrementTime"]] = df["TimeControl"].str.split("+", expand=True)
df["BaseTime"] = df["BaseTime"].astype(np.int16)
df["IncrementTime"] = df["IncrementTime"].astype(np.int16)

In [19]:
df.head()

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,chess_moves_list,evaluations_list,times_list
0,Rated Bullet game,https://lichess.org/PnoVpOEg,MaggieSimpson,bcleary2000,0-1,2107,2164,-9.0,9.0,A80,Dutch Defense,60+0,Normal,"['d4', 'f5', 'Nf3', 'd5', 'g3', 'Nf6', 'Bg2', ...","['0.08', '0.47', '0.22', '0.7', '0.62', '0.37'...","['0:01:00', '0:01:00', '0:01:00', '0:00:59', '..."
1,Rated Bullet game,https://lichess.org/9nDHcoZY,fred314,Rlimab,1-0,1220,1054,6.0,-13.0,B23,Sicilian Defense: Closed Variation,60+0,Time forfeit,"['e4', 'c5', 'Nc3', 'a6', 'd4', 'e6', 'dxc5', ...","['0.09', '0.17', '0.0', '0.08', '0.0', '0.54',...","['0:01:00', '0:01:00', '0:00:59', '0:00:59', '..."
2,Rated Classical game,https://lichess.org/JqsK8a3O,Marco74,TheMeerkat,0-1,1475,1614,-7.0,51.0,D01,Queen's Pawn Game: Chigorin Variation,1500+10,Normal,"['d4', 'Nf6', 'Nc3', 'd5', 'Nf3', 'e6', 'a3', ...","['0.08', '0.13', '-0.06', '-0.12', '-0.21', '-...","['0:25:00', '0:25:00', '0:25:07', '0:25:05', '..."
3,Rated Standard game,https://lichess.org/ciXTg6ei,Acemoneyg,ScorchedToast,0-1,964,951,-11.0,12.0,C44,King's Knight Opening: Normal Variation,600+0,Normal,"['e4', 'e5', 'Nf3', 'Nc6', 'b3', 'Nf6', 'Bb2',...","['0.09', '0.23', '0.25', '0.26', '-0.25', '0.0...","['0:10:00', '0:10:00', '0:09:59', '0:09:56', '..."
4,Rated Bullet game,https://lichess.org/74YofaHW,Radrax,Gommo,0-1,1170,1136,-27.0,41.0,A40,Queen's Pawn,120+1,Time forfeit,"['d4', 'a6', 'Bf4', 'b5', 'Nc3', 'Bb7', 'Nf3',...","['0.08', '0.46', '0.36', '0.79', '0.15', '0.14...","['0:02:00', '0:02:00', '0:02:00', '0:01:59', '..."


In [106]:
def time_str_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s


def calc_game_duration(time_and_increment):
    time_list, increment_time = time_and_increment.values
    increment = (len(times) - 2) * increment_time
    return sum(time_list[:2]) - sum(time_list[-2:]) + increment


def parse_time_list_to_seconds(time_list_str):
    time_list = time_list_str[2:-2].split("', '")
    return [time_str_to_seconds(time_str) for time_str in time_list]

In [31]:
%%time
df["time_in_seconds"] = df["times_list"].apply(parse_time_list_to_seconds)

CPU times: user 4.6 s, sys: 52.1 ms, total: 4.65 s
Wall time: 4.67 s


In [109]:
df["total_time"] = df[["time_in_seconds", "IncrementTime"]].apply(calc_game_duration, axis=1)

In [110]:
df.head()

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,chess_moves_list,evaluations_list,times_list,time_in_seconds,total_time,BaseTime,IncrementTime
0,Rated Bullet game,https://lichess.org/PnoVpOEg,MaggieSimpson,bcleary2000,0-1,2107,2164,-9.0,9.0,A80,Dutch Defense,60+0,Normal,"['d4', 'f5', 'Nf3', 'd5', 'g3', 'Nf6', 'Bg2', ...","['0.08', '0.47', '0.22', '0.7', '0.62', '0.37'...","['0:01:00', '0:01:00', '0:01:00', '0:00:59', '...","[60, 60, 60, 59, 59, 59, 59, 59, 59, 58, 59, 5...",108,60,0
1,Rated Bullet game,https://lichess.org/9nDHcoZY,fred314,Rlimab,1-0,1220,1054,6.0,-13.0,B23,Sicilian Defense: Closed Variation,60+0,Time forfeit,"['e4', 'c5', 'Nc3', 'a6', 'd4', 'e6', 'dxc5', ...","['0.09', '0.17', '0.0', '0.08', '0.0', '0.54',...","['0:01:00', '0:01:00', '0:00:59', '0:00:59', '...","[60, 60, 59, 59, 58, 58, 56, 57, 53, 54, 51, 5...",106,60,0
2,Rated Classical game,https://lichess.org/JqsK8a3O,Marco74,TheMeerkat,0-1,1475,1614,-7.0,51.0,D01,Queen's Pawn Game: Chigorin Variation,1500+10,Normal,"['d4', 'Nf6', 'Nc3', 'd5', 'Nf3', 'e6', 'a3', ...","['0.08', '0.13', '-0.06', '-0.12', '-0.21', '-...","['0:25:00', '0:25:00', '0:25:07', '0:25:05', '...","[1500, 1500, 1507, 1505, 1509, 1510, 1517, 151...",836,1500,10
3,Rated Standard game,https://lichess.org/ciXTg6ei,Acemoneyg,ScorchedToast,0-1,964,951,-11.0,12.0,C44,King's Knight Opening: Normal Variation,600+0,Normal,"['e4', 'e5', 'Nf3', 'Nc6', 'b3', 'Nf6', 'Bb2',...","['0.09', '0.23', '0.25', '0.26', '-0.25', '0.0...","['0:10:00', '0:10:00', '0:09:59', '0:09:56', '...","[600, 600, 599, 596, 592, 588, 591, 584, 589, ...",240,600,0
4,Rated Bullet game,https://lichess.org/74YofaHW,Radrax,Gommo,0-1,1170,1136,-27.0,41.0,A40,Queen's Pawn,120+1,Time forfeit,"['d4', 'a6', 'Bf4', 'b5', 'Nc3', 'Bb7', 'Nf3',...","['0.08', '0.46', '0.36', '0.79', '0.15', '0.14...","['0:02:00', '0:02:00', '0:02:00', '0:01:59', '...","[120, 120, 120, 119, 118, 118, 116, 117, 112, ...",212,120,1


In [57]:
data = df.iloc[[2]][["TimeControl", "times_list", "time_in_seconds", "total_time", "BaseTime", "IncrementTime"]]

In [59]:
data["time_in_seconds"].values

array([list([1500, 1500, 1507, 1505, 1509, 1510, 1517, 1515, 1517, 1513, 1514, 1515, 1509, 1515, 1498, 1510, 1501, 1510, 1506, 1515, 1511, 1516, 1485, 1519, 1459, 1524, 1414, 1526, 1388, 1530, 1302, 1533, 1304, 1534, 1266, 1479, 1139, 1472, 1142, 1475, 1127, 1471, 1108, 1476])],
      dtype=object)

In [105]:
times = [1500, 1500, 1507, 1505, 1509, 1510, 1517, 1515, 1517, 1513, 1514, 1515, 1509, 1515, 1498, 1510, 1501, 1510, 1506, 1515, 1511, 1516, 1485, 1519, 1459, 1524, 1414, 1526, 1388, 1530, 1302, 1533, 1304, 1534, 1266, 1479, 1139, 1472, 1142, 1475, 1127, 1471, 1108, 1476]

(len(times) - 2) * 10

420